In [1]:
import cv2 
import numpy as np
import os
import json
import random
import pandas as pd
import matplotlib.pyplot as plt
import PIL
from PIL import Image

In [2]:
local = True
note = True

In [3]:
if local:
    if note:
        data_path = 'G:/.shortcut-targets-by-id/13rzmaQd1P_XCr1avulFNSGjj0WZFQsrj/BDHansen_Local'
        # workspace_path = 'C:/Users/lucas/GoogleDrive/.shortcut-targets-by-id/1lAkgsgLg50ktk5RH220KX7aLsltKY_SP/Trabalho Jacks/skinreadermodels'
    else:  
        data_path = 'C:/Users/lucas/Documents/Area_de_Trabalho/about-skinreader/dataset/BDHansen'
        workspace_path = 'C:/Users/lucas/GoogleDrive/.shortcut-targets-by-id/1lAkgsgLg50ktk5RH220KX7aLsltKY_SP/Trabalho Jacks/skinreadermodels'
else:
    data_path = '/content/drive/MyDrive/Colab Notebooks/dataset/skinreader/BDHansen'
    workspace_path = '/content/drive/MyDrive/Colab Notebooks/skinreadermodels'

# info_path = os.path.join(workspace_path, 'info')
images_closeup_path = os.path.join(data_path, 'closeup')
metadata_path = os.path.join(data_path, 'metadados')
images_all_path = os.path.join(data_path, 'all_closeup_panoramic')
images_panoramic_path = os.path.join(data_path, 'panoramic')
save_path_dataaumented = os.path.join(data_path, 'augmented_cloaseup')

In [4]:
contours = {}

In [5]:
def fixFileName(str_ret):
    return "".join([str_ret, '.jpg'])

def load_jsonfile(json_path):
    with open(json_path,'r') as arquivo:
        paciente = arquivo.read()
        return json.loads(paciente)
    
def get_json_files():
    files = os.listdir(metadata_path)
    return [load_jsonfile(os.path.join(metadata_path, f)) for f in files if f.endswith('.json')]

def get_data(json_list):
    patient_id_list = []
    image_list_for_patient = []
    for j in json_list:
        patient_id_list.append(j['_via_settings']['project']['name'].split('/')[0])
        list_aux = []
        for key, value in j['_via_img_metadata'].items():
            regions = []
            if len(value['regions']) != 0 :
                # print(value['filename'])
                # print(value['regions'][0])
                #* Em uma imagem odas as regiões teem o mesmo diagnóstico? 
                list_aux.append((value['filename'], value['regions'][0]['region_attributes']['leprosy'], len(value['regions'])))
                for region in value['regions']:
                    regions.append([region['shape_attributes']['all_points_x'], region['shape_attributes']['all_points_y']])
                
                contours[value['filename']] = regions
            else:
                print("Paciente com imagem sem regiões demarcadas:\nPaciente: {}\nImagem: {}".format(j["_via_settings"]["project"]["name"], value["filename"]))    
        if len(j['_via_img_metadata'].items()) == 0:
            print("Paciente sem imagens associadas!\nPaciente: {}".format(j["_via_settings"]["project"]["name"]))
        if len(list_aux) != 0: image_list_for_patient.append(list_aux) #lista de imagens e diagnóstico para cada imagem para cada paciente
    
    dados = []
    c_imagenames = os.listdir(images_closeup_path)
    a_imagenames = os.listdir(images_all_path)
    p_imagenames = os.listdir(images_panoramic_path)
    print(f'Numero de imagens -> \nColseup: {len(c_imagenames)}\nAll: {len(a_imagenames)}\nPanoramicas: {len(p_imagenames)}')
    
    for index, p in enumerate(patient_id_list):
        study_id = p.split('-')[1]

        c_images = [image_name for image_name in c_imagenames if study_id in image_name]
        a_images = [image_name for image_name in a_imagenames if study_id in image_name]
        p_images = [image_name for image_name in p_imagenames if study_id in image_name]

        for i in c_images:
            no_extention = i.split('.')[0]
            for n in image_list_for_patient[index]:
                if no_extention in n[0]:
                    dados.append([
                        study_id, 
                        p, 
                        no_extention, 
                        n[0], 
                        os.path.join(images_closeup_path, i), 
                        n[1], 
                        'closeup', 
                        i.split('.')[0].split('-')[-1],
                        n[2]
                        ])

    return dados

In [6]:
jsonfiles = get_json_files()
print(f'Patients Num -> {len(jsonfiles)}')
data = get_data(jsonfiles)
print(f'Imagens Num -> {len(data)}')

Patients Num -> 225
Paciente com imagem sem regiões demarcadas:
Paciente: Patient-014IS/
Imagem: https://patientimage.blob.core.windows.net/processedimages/Patient-014IS/Image-014IS2-ulimbs-panoramic.jpg
Paciente com imagem sem regiões demarcadas:
Paciente: Patient-080JS/
Imagem: https://patientimage.blob.core.windows.net/processedimages/Patient-080JS/Image-080JS1-trunk-panoramic.jpg
Numero de imagens -> 
Colseup: 733
All: 1232
Panoramicas: 500
Imagens Num -> 717


## Head and Tail

In [16]:
df = pd.DataFrame(data, columns=['StudyID', 'patient_id', 'image_name', 'filename', 'image_path', 'patient_leprosy', 'local_dist', 'type', 'num_regions'])
df.head(25)

,StudyID,patient_id,image_name,filename,image_path,patient_leprosy,local_dist,type,num_regions
0,004MF,Patient-004MF,Image-004MF-llimbs-closeup,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/13rzmaQd1P_XCr1avul...,other_dermotosis,closeup,closeup,1
1,006CJ,Patient-006CJ,Image-006CJ1-trunk-edge,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/13rzmaQd1P_XCr1avul...,leprosy,closeup,edge,1
2,006CJ,Patient-006CJ,Image-006CJ1-trunk-closeup,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/13rzmaQd1P_XCr1avul...,leprosy,closeup,closeup,1
3,002IR,Patient-002IR,Image-002IR2-ulimbs-closeup,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/13rzmaQd1P_XCr1avul...,leprosy,closeup,closeup,1
4,008JS,Patient-008JS,Image-008JS2-ulimbs-closeup,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/13rzmaQd1P_XCr1avul...,leprosy,closeup,closeup,15
5,008JS,Patient-008JS,Image-008JS3-ulimbs-closeup,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/13rzmaQd1P_XCr1avul...,leprosy,closeup,closeup,5
6,008JS,Patient-008JS,Image-008JS1-trunk-closeup,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/13rzmaQd1P_XCr1avul...,leprosy,closeup,closeup,1
7,001WN,Patient-001WN,Image-001WN3-ulimbs-closeup,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/13rzmaQd1P_XCr1avul...,leprosy,closeup,closeup,1
8,001WN,Patient-001WN,Image-001WN2-trunk-closeup,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/13rzmaQd1P_XCr1avul...,leprosy,closeup,closeup,1
9,007GB,Patient-007GB,Image-007GB1-ulimbs-closeup,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/13rzmaQd1P_XCr1avul...,other_dermotosis,closeup,closeup,1


In [17]:
df.tail(25)

,StudyID,patient_id,image_name,filename,image_path,patient_leprosy,local_dist,type,num_regions
692,216MG,Patient-216MG,Image-216MG1-llimbs-closeup,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/13rzmaQd1P_XCr1avul...,other_dermotosis,closeup,closeup,2
693,216MG,Patient-216MG,Image-216MG1-llimbs-edge,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/13rzmaQd1P_XCr1avul...,other_dermotosis,closeup,edge,1
694,224JQ,Patient-224JQ,Image-224JQ2-trunk-closeup,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/13rzmaQd1P_XCr1avul...,leprosy,closeup,closeup,1
695,224JQ,Patient-224JQ,Image-224JQ1-trunk-edge,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/13rzmaQd1P_XCr1avul...,leprosy,closeup,edge,1
696,224JQ,Patient-224JQ,Image-224JQ1-trunk-closeup,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/13rzmaQd1P_XCr1avul...,leprosy,closeup,closeup,3
697,224JQ,Patient-224JQ,Image-224JQ2-trunk-edge,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/13rzmaQd1P_XCr1avul...,leprosy,closeup,edge,1
698,227ER,Patient-227ER,Image-227ER1-ulimbs-closeup,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/13rzmaQd1P_XCr1avul...,other_dermotosis,closeup,closeup,1
699,227ER,Patient-227ER,Image-227ER2-ulimbs-closeup,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/13rzmaQd1P_XCr1avul...,other_dermotosis,closeup,closeup,1
700,228FM,Patient-228FM,Image-228FM2-face-closeup,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/13rzmaQd1P_XCr1avul...,leprosy,closeup,closeup,1
701,228FM,Patient-228FM,Image-228FM5-llimbs-closeup,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/13rzmaQd1P_XCr1avul...,leprosy,closeup,closeup,1


In [18]:
closeup = 'closeup'
print(f'Num imagens: {len(df)}')
print(f'Num closeup imagens: {len(df.loc[df.local_dist == closeup])}')
print(f'Num imagens with more than one regions: {len(df.loc[df.num_regions >= 2])}')
print(f'Num imagens with equal to one regions: {len(df.loc[df.num_regions < 2])}')
print(f'Num patients: {len(df.patient_id.unique())}')
print(f'Num patients that have imagens with more than one regions: {len(df.loc[df.num_regions >= 2].patient_id.unique())}')
print(f'Num patients that have imagens with equal to one regions: {len(df.loc[df.num_regions < 2].patient_id.unique())}')

Num imagens: 717
Num closeup imagens: 717
Num imagens with more than one regions: 232
Num imagens with equal to one regions: 485
Num patients: 218
Num patients that have imagens with more than one regions: 116
Num patients that have imagens with equal to one regions: 194


# Testes sobre os pacientes
* Existe imagens do mesmo paciente mas com diagnósticos difetentes?
* O tipo de imagem closeup engloba: closeup e edge

In [19]:
for index, study_id in enumerate(df['StudyID'].unique()):
    list_diagnostics = df.loc[df['image_name'].str.contains(study_id), 'patient_leprosy'].values
    
    aux = list_diagnostics[0]
    cont_diff = 0
    for diagnostic in list_diagnostics:
        if diagnostic != aux:
            print(f'[ATENTION] Different Diagnostic for the same patient, considering differents images!!!\nDiagnostic list for images: {list_diagnostics}')
            cont_diff += 1
    
    if cont_diff != 0:
        print("Different!!")
        # print(f'[OK]Same diagnostic for all images, for the patient: {index}: {study_id}\nDiagnostic list for images: {list_diagnostics}')

In [20]:
df_closeup = df.loc[df['type'] != 'panoramic']
df_closeup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 717 entries, 0 to 716
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   StudyID          717 non-null    object
 1   patient_id       717 non-null    object
 2   image_name       717 non-null    object
 3   filename         717 non-null    object
 4   image_path       717 non-null    object
 5   patient_leprosy  717 non-null    object
 6   local_dist       717 non-null    object
 7   type             717 non-null    object
 8   num_regions      717 non-null    int64 
dtypes: int64(1), object(8)
memory usage: 56.0+ KB


# Loop para pre-processar e destacar as regiões

In [21]:
def get_dictContours(image, list_contours):
    dict_contours = {}

    for index, region in enumerate(list_contours):
        pts = np.array(tuple(zip(region[0], region[1])), dtype=np.int32)
        
        x,y,w,h = cv2.boundingRect(pts)
        
        text = 'reg_' + str(index)
        dict_contours[text] = [x,y,w,h]
    
    return dict_contours 

In [22]:
def isInside(box1, box2):
  
  box1_xmin = box1[0]
  box1_xmax = box1[0] + box1[2]
  box1_ymin = box1[1]
  box1_ymax = box1[1] + box1[3]
  
  box2_xmin = box2[0]
  box2_xmax = box2[0] + box2[2]
  box2_ymin = box2[1]
  box2_ymax = box2[1] + box2[3]
  
  # print(box1_xmax, box1_xmin, box1_ymax, box1_ymin)
  # print(box2_xmax, box2_xmin, box2_ymax, box2_ymin)
  
  in_range_along_y_1 = box1_ymin < box2_ymax and box1_ymin > box2_ymin
  in_range_along_y_2 = box1_ymax < box2_ymax and box1_ymax > box2_ymin
  
  in_range_along_x_1 = box1_xmin < box2_xmax and box1_xmin > box2_xmin
  in_range_along_x_2 = box1_xmax < box2_xmax and box1_xmax > box2_xmin
  
  isInside = (in_range_along_y_1 and (in_range_along_x_1 or in_range_along_x_2)) or (in_range_along_y_2 and (in_range_along_x_1 or in_range_along_x_2))
  
  # print(in_range_along_x_1, in_range_along_x_2, in_range_along_y_1, in_range_along_y_2)
  
  if isInside:
    # print('Box Inside')
    return True                      
  
  return False

In [23]:
def group_overlaped_regions(dict_contours, image):

    image_gray_reference = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2GRAY)

    while True:
        reiniciar = False
        found = False
        for index, key_1 in enumerate(dict_contours.keys()):
            
            if (index + 1) < len(dict_contours):
                
                for key_2 in list(dict_contours.keys())[index + 1:]:
                    
                    # print(key_1, key_2)
                    box1_in_box2 = isInside(dict_contours[key_1], dict_contours[key_2])
                    box2_in_box1 = isInside(dict_contours[key_2], dict_contours[key_1])
                    # print('\n\n')
                    
                    if box1_in_box2 or box2_in_box1:
                        # print(box1_in_box2, box2_in_box1)
                        # print(f'region 1: {key_1}, {dict_contours[key_1]}')
                        # print(f'region 2: {key_2}, {dict_contours[key_2]}') 
                        # print('\n\n')
                        
                        black = np.zeros(image_gray_reference.shape, dtype='uint8')
                    
                        mask_1 = cv2.rectangle(black.copy(), (dict_contours[key_1][0], dict_contours[key_1][1]), (dict_contours[key_1][0]+dict_contours[key_1][2],dict_contours[key_1][1]+dict_contours[key_1][3]), (255, 255, 255), -1)
                        mask1_area = dict_contours[key_1][2] * dict_contours[key_1][3]
                        
                        mask_2 = cv2.rectangle(black.copy(), (dict_contours[key_2][0], dict_contours[key_2][1]), (dict_contours[key_2][0]+dict_contours[key_2][2],dict_contours[key_2][1]+dict_contours[key_2][3]), (255, 255, 255), -1)
                        mask2_area = dict_contours[key_2][2] * dict_contours[key_2][3]
                        
                        intersection = cv2.bitwise_and(mask_1, mask_2)
                        
                        if intersection.any():
                            
                            intersection = np.where(intersection, 255, 0)
                            
                            intersection_contours, _ = cv2.findContours(np.uint8(intersection), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
                            intersection_area = cv2.contourArea(intersection_contours[0])
                            
                            # print(f'Area: {mask1_area}')
                            # print(f'Area: {mask2_area}')
                            # print(f'Area: {intersection_area}')
                            # print(f'Conta: {min(mask1_area, mask2_area) / 2}')
                            if (min(mask1_area, mask2_area) / 2) < intersection_area:
                                # print('O contorno menor deve estar dentro do contorno maior!')
                                
                                newx, newy, neww, newh = cv2.boundingRect(np.array([
                                    [min(dict_contours[key_1][0], dict_contours[key_2][0]), min(dict_contours[key_1][1], dict_contours[key_2][1])],
                                    [min(dict_contours[key_1][0], dict_contours[key_2][0]), max(dict_contours[key_1][1] + dict_contours[key_1][3], dict_contours[key_2][1] + dict_contours[key_2][3])],
                                    [max(dict_contours[key_1][0] + dict_contours[key_1][2], dict_contours[key_2][0] + dict_contours[key_2][2]), min(dict_contours[key_1][1], dict_contours[key_2][1])],
                                    [max(dict_contours[key_1][0] + dict_contours[key_1][2], dict_contours[key_2][0] + dict_contours[key_2][2]), max(dict_contours[key_1][1] + dict_contours[key_1][3], dict_contours[key_2][1] + dict_contours[key_2][3])]
                                    
                                ], dtype=np.int32))
                                
                                del dict_contours[key_1]
                                del dict_contours[key_2]
                                
                                dict_contours[key_1+key_2] = [newx, newy, neww, newh]
                                reiniciar = True
                                found = True
                                break 
            else:
                break   
            
            if reiniciar: break
        
        if not found:
            break
    
    return dict_contours

In [24]:
def group_small_images(dict_contours, dimension):
    while True:
        found = False
        for key in sorted(dict_contours, key = lambda region: dict_contours[region][2] * dict_contours[region][3]):
            
            if dict_contours[key][2] < dimension[0] or dict_contours[key][3] < dimension[1]:
                # print('Dimensão da região menor do que 300x300 pixels')
                
                val_min = 999999999
                chousen_reg = ''
                for key_1 in sorted(dict_contours, key = lambda region: dict_contours[region][2] * dict_contours[region][3]):
                    if key_1 != key:
                        center_key_x = dict_contours[key][0] + (dict_contours[key][2] / 2)
                        center_key_y = dict_contours[key][1] + (dict_contours[key][3] / 2)
                        center_key1_x = dict_contours[key_1][0] + (dict_contours[key_1][2] / 2)
                        center_key1_y = dict_contours[key_1][1] + (dict_contours[key_1][3] / 2)
                                
                        dist =  np.sqrt((center_key_x - center_key1_x)**2 + (center_key_y - center_key1_y)**2)
                        
                        # print(dist, val_min)
                        if dist < val_min:
                            # print(val, val_min)
                            val_min = dist
                            chousen_reg = key_1
                
                # print(chousen_reg)
                # print(dict_contours[key])
                # print(dict_contours[chousen_reg])
                # print('\n\n')
                
                newx, newy, neww, newh = cv2.boundingRect(np.array([
                    [min(dict_contours[key][0], dict_contours[chousen_reg][0]), min(dict_contours[key][1], dict_contours[chousen_reg][1])],
                    [min(dict_contours[key][0], dict_contours[chousen_reg][0]), max(dict_contours[key][1] + dict_contours[key][3], dict_contours[chousen_reg][1] + dict_contours[chousen_reg][3])],
                    [max(dict_contours[key][0] + dict_contours[key][2], dict_contours[chousen_reg][0] + dict_contours[chousen_reg][2]), min(dict_contours[key][1], dict_contours[chousen_reg][1])],
                    [max(dict_contours[key][0] + dict_contours[key][2], dict_contours[chousen_reg][0] + dict_contours[chousen_reg][2]), max(dict_contours[key][1] + dict_contours[key][3], dict_contours[chousen_reg][1] + dict_contours[chousen_reg][3])]
                ], dtype=np.int32))
                
                del dict_contours[key]
                del dict_contours[chousen_reg]
            
                dict_contours[key+chousen_reg] = [newx, newy, neww, newh]
                found = True
                break
        if not found:
            break
    return dict_contours

In [25]:
def expand_regions(dict_contours, expand, image):
    
    for key in dict_contours: 
        # x
        x  = dict_contours[key][0] - expand if (dict_contours[key][0] >= expand) else 0
        # y
        y = dict_contours[key][1] - expand if (dict_contours[key][1] >= expand) else 0
        # w
        w = dict_contours[key][2] + expand * 2 if (dict_contours[key][0] + dict_contours[key][2] + expand <= image.shape[1]) else image.shape[1] - dict_contours[key][0] + dict_contours[key][2]
        # h
        h = dict_contours[key][3] + expand * 2 if (dict_contours[key][1] + dict_contours[key][3] + expand <= image.shape[0]) else image.shape[0] - dict_contours[key][1] + dict_contours[key][3]
        
        dict_contours[key] = [x, y, w, h]
        
    return dict_contours

In [23]:
def get_laplacian_variance(image):
	# compute the Laplacian of the image and then return the focus
	# measure, which is simply the variance of the Laplacian
	return cv2.Laplacian(image, cv2.CV_64F).var()

## Interate over rows

In [24]:
df_regions = df.loc[df['num_regions'] >= 2]
list_dict = []
list_check = []
save_path_regions = os.path.join(data_path, 'closeup_regions')

In [25]:
print(f'Num imagens: {len(df)}')
print(f'Num closeup imagens: {len(df.loc[df.local_dist == closeup])}')
print(f'Num imagens with more than one regions: {len(df.loc[df.num_regions >= 2])}')
print(f'Num imagens with equal to one regions: {len(df.loc[df.num_regions < 2])}')
print(f'Num patients: {len(df.patient_id.unique())}')
print(f'Num patients that have imagens with more than one regions: {len(df.loc[df.num_regions >= 2].patient_id.unique())}')
print(f'Num patients that have imagens with equal to one regions: {len(df.loc[df.num_regions < 2].patient_id.unique())}')

Num imagens: 717
Num closeup imagens: 717
Num imagens with more than one regions: 232
Num imagens with equal to one regions: 485
Num patients: 218
Num patients that have imagens with more than one regions: 116
Num patients that have imagens with equal to one regions: 194


In [26]:
print(f'Num imagens: {len(df)}')
print(f'Num closeup imagens: {len(df.loc[df.local_dist == closeup])}')
print(f'Num imagens with more than one regions: {len(df_regions.loc[df_regions.num_regions >= 2])}')
print(f'Num imagens with equal to one regions: {len(df.loc[df.num_regions < 2])}')
print(f'Num patients: {len(df.patient_id.unique())}')
print(f'Num patients that have imagens with more than one regions: {len(df_regions.loc[df_regions.num_regions >= 2].patient_id.unique())}')
print(f'Num patients that have imagens with equal to one regions: {len(df.loc[df.num_regions < 2].patient_id.unique())}')

Num imagens: 717
Num closeup imagens: 717
Num imagens with more than one regions: 232
Num imagens with equal to one regions: 485
Num patients: 218
Num patients that have imagens with more than one regions: 116
Num patients that have imagens with equal to one regions: 194


In [27]:
for index, row in df_regions.iterrows():
    image = cv2.imread(row['image_path'])
    # image_rgb = np.copy(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
    dict_contours = get_dictContours(image, contours[row['filename']])
    
    dict_contours = group_overlaped_regions(dict_contours=dict_contours, image=image)
    
    dict_contours = group_small_images(dict_contours=dict_contours, dimension=[300,300])
    
    dict_contours = expand_regions(dict_contours=dict_contours, expand= 100, image=image)
    
    for key in dict_contours.keys():
        roi_name = row['image_name'] + '_' + key.replace('_', '')
        
        
        y = dict_contours[key][1] if dict_contours[key][1] >= 0 else 0
        h = dict_contours[key][3] if dict_contours[key][3] >= 0 else abs(dict_contours[key][3])
        x = dict_contours[key][0] if dict_contours[key][0] >= 0 else 0
        w = dict_contours[key][2] if dict_contours[key][2] >= 0 else abs(dict_contours[key][2])
        
        x = x + 10
        
        roi = np.copy(image[y : y + h, x : x + w])
        gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        focous_mesure = get_laplacian_variance(gray_roi)
        
        if (focous_mesure < 5):
            roi_path = os.path.join(save_path_regions, 'blurry', roi_name + '.jpg')
        else:
            roi_path = os.path.join(save_path_regions, roi_name + '.jpg')
                    
        list_check.append(cv2.imwrite(roi_path, roi))
        
        # print(roi_path)
        # print(focous_mesure)
        
        list_dict.append({
            'StudyID': row['StudyID'],
            'patient_id': row['patient_id'],
            'image_name': roi_name,
            'filename': row['filename'],
            'image_path': roi_path,
            'patient_leprosy': row['patient_leprosy'],
            'local_dist': 'closep_regions',
            'type': 'closeup_regions',
            'num_regions': 1
        })

In [28]:
t = f = 0
for b in list_check:
    if b : t+=1
    else: f+=1
print(f'True: {t}\nFalse: {f}')

True: 802
False: 0


In [40]:
df_of_interest = df.loc[df['num_regions'] < 2]
# len(df_of_interest.patient_id.unique())

In [41]:
print(f'Num imagens: {len(df)}')
print(f'Num closeup imagens: {len(df.loc[df.local_dist == closeup])}')
print(f'Num imagens with more than one regions: {len(df.loc[df.num_regions >= 2])}')
print(f'Num imagens with equal to one regions: {len(df.loc[df.num_regions < 2])}')
print(f'Num patients: {len(df.patient_id.unique())}')
print(f'Num patients that have imagens with more than one regions: {len(df.loc[df.num_regions >= 2].patient_id.unique())}')
print(f'Num patients that have imagens with equal to one regions: {len(df.loc[df.num_regions < 2].patient_id.unique())}')

Num imagens: 717
Num closeup imagens: 717
Num imagens with more than one regions: 232
Num imagens with equal to one regions: 485
Num patients: 218
Num patients that have imagens with more than one regions: 116
Num patients that have imagens with equal to one regions: 194


In [42]:
print(f'Num imagens: {len(df)}')
print(f'Num closeup imagens: {len(df.loc[df.local_dist == closeup])}')

print(f'Num imagens with more than one regions: {len(df_regions.loc[df_regions.num_regions >= 2])}')
print(f'Num imagens with equal to one regions: {len(df_of_interest.loc[df_of_interest.num_regions < 2])}')

print(f'Num patients: {len(df.patient_id.unique())}')
print(f'Num patients that have imagens with more than one regions: {len(df_regions.loc[df_regions.num_regions >= 2].patient_id.unique())}')
print(f'Num patients that have imagens with equal to one regions: {len(df_of_interest.loc[df_of_interest.num_regions < 2].patient_id.unique())}')

Num imagens: 717
Num closeup imagens: 717
Num imagens with more than one regions: 232
Num imagens with equal to one regions: 485
Num patients: 218
Num patients that have imagens with more than one regions: 116
Num patients that have imagens with equal to one regions: 194


In [43]:
df_of_interest = pd.concat([df_of_interest, pd.DataFrame(list_dict)], ignore_index=True)

In [44]:
df_of_interest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1287 entries, 0 to 1286
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   StudyID          1287 non-null   object
 1   patient_id       1287 non-null   object
 2   image_name       1287 non-null   object
 3   filename         1287 non-null   object
 4   image_path       1287 non-null   object
 5   patient_leprosy  1287 non-null   object
 6   local_dist       1287 non-null   object
 7   type             1287 non-null   object
 8   num_regions      1287 non-null   int64 
dtypes: int64(1), object(8)
memory usage: 90.6+ KB


In [45]:
df_of_interest.head()

,StudyID,patient_id,image_name,filename,image_path,patient_leprosy,local_dist,type,num_regions
0,006CJ,Patient-006CJ,Image-006CJ1-trunk-edge,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,leprosy,closeup,edge,1
1,006CJ,Patient-006CJ,Image-006CJ1-trunk-closeup,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,leprosy,closeup,closeup,1
2,002IR,Patient-002IR,Image-002IR2-ulimbs-closeup,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,leprosy,closeup,closeup,1
3,004MF,Patient-004MF,Image-004MF-llimbs-closeup,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,other_dermotosis,closeup,closeup,1
4,008JS,Patient-008JS,Image-008JS1-trunk-closeup,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,leprosy,closeup,closeup,1


In [46]:
df_of_interest.tail()

,StudyID,patient_id,image_name,filename,image_path,patient_leprosy,local_dist,type,num_regions
1282,223LO,Patient-223LO,Image-223LO1-ulimbs-closeup_reg2reg1,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,other_dermotosis,closep_regions,closeup_regions,1
1283,226FJ,Patient-226FJ,Image-226FJ3-ulimbs-closeup_reg0,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,leprosy,closep_regions,closeup_regions,1
1284,226FJ,Patient-226FJ,Image-226FJ3-ulimbs-closeup_reg1,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,leprosy,closep_regions,closeup_regions,1
1285,226FJ,Patient-226FJ,Image-226FJ3-ulimbs-closeup_reg2,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,leprosy,closep_regions,closeup_regions,1
1286,226FJ,Patient-226FJ,Image-226FJ3-ulimbs-closeup_reg4reg3,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,leprosy,closep_regions,closeup_regions,1


In [47]:
print(f'Num imagens: {len(df)}')
print(f'Num closeup imagens: {len(df.loc[df.local_dist == closeup])}')

print(f'Num imagens with more than one regions: {len(df_of_interest.loc[df_of_interest.num_regions >= 2])}')

print(f'Num patients: {len(df.patient_id.unique())}')
print(f'Num patients that have imagens with equal to one regions: {len(df_of_interest.loc[df_of_interest.num_regions < 2].patient_id.unique())}')

Num imagens: 717
Num closeup imagens: 717
Num imagens with more than one regions: 0
Num patients: 218
Num patients that have imagens with equal to one regions: 218


In [48]:
df_of_interest.to_csv(os.path.join(save_path_regions, 'info_regions.csv'))

In [49]:
df_of_interest.head(10)

,StudyID,patient_id,image_name,filename,image_path,patient_leprosy,local_dist,type,num_regions
0,006CJ,Patient-006CJ,Image-006CJ1-trunk-edge,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,leprosy,closeup,edge,1
1,006CJ,Patient-006CJ,Image-006CJ1-trunk-closeup,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,leprosy,closeup,closeup,1
2,002IR,Patient-002IR,Image-002IR2-ulimbs-closeup,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,leprosy,closeup,closeup,1
3,004MF,Patient-004MF,Image-004MF-llimbs-closeup,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,other_dermotosis,closeup,closeup,1
4,008JS,Patient-008JS,Image-008JS1-trunk-closeup,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,leprosy,closeup,closeup,1
5,001WN,Patient-001WN,Image-001WN3-ulimbs-closeup,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,leprosy,closeup,closeup,1
6,001WN,Patient-001WN,Image-001WN2-trunk-closeup,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,leprosy,closeup,closeup,1
7,005MO,Patient-005MO,Image-005MO-trunk-closeup,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,other_dermotosis,closeup,closeup,1
8,007GB,Patient-007GB,Image-007GB1-ulimbs-closeup,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,other_dermotosis,closeup,closeup,1
9,014IS,Patient-014IS,Image-014IS2-ulimbs-closeup,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,leprosy,closeup,closeup,1


In [50]:
df_of_interest.tail(10)

,StudyID,patient_id,image_name,filename,image_path,patient_leprosy,local_dist,type,num_regions
1277,224JQ,Patient-224JQ,Image-224JQ1-trunk-closeup_reg0,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,leprosy,closep_regions,closeup_regions,1
1278,224JQ,Patient-224JQ,Image-224JQ1-trunk-closeup_reg1,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,leprosy,closep_regions,closeup_regions,1
1279,224JQ,Patient-224JQ,Image-224JQ1-trunk-closeup_reg2,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,leprosy,closep_regions,closeup_regions,1
1280,223LO,Patient-223LO,Image-223LO1-ulimbs-closeup_reg3,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,other_dermotosis,closep_regions,closeup_regions,1
1281,223LO,Patient-223LO,Image-223LO1-ulimbs-closeup_reg4reg0,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,other_dermotosis,closep_regions,closeup_regions,1
1282,223LO,Patient-223LO,Image-223LO1-ulimbs-closeup_reg2reg1,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,other_dermotosis,closep_regions,closeup_regions,1
1283,226FJ,Patient-226FJ,Image-226FJ3-ulimbs-closeup_reg0,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,leprosy,closep_regions,closeup_regions,1
1284,226FJ,Patient-226FJ,Image-226FJ3-ulimbs-closeup_reg1,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,leprosy,closep_regions,closeup_regions,1
1285,226FJ,Patient-226FJ,Image-226FJ3-ulimbs-closeup_reg2,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,leprosy,closep_regions,closeup_regions,1
1286,226FJ,Patient-226FJ,Image-226FJ3-ulimbs-closeup_reg4reg3,https://patientimage.blob.core.windows.net/pro...,G:/.shortcut-targets-by-id/1ZFZCmSDxnFF_GzgF2w...,leprosy,closep_regions,closeup_regions,1


In [46]:
len(list_dict)

802